### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [ ]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [2]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor

from pathlib import Path
import pandas as pd
import logging


logger = logging.getLogger()
device_cpu = torch.device("cpu")

In [3]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [3]:
df = pd.read_csv('./data/AA_xml_manul - 0-200_0-403.csv')

In [4]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 80
Nombre de commentaires dans le train_set: 322


In [5]:
labels = df.columns[2:4].to_list() 
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [4]:
df_texts = pd.read_csv('./data/raw_xml_bsv_0-200.csv')

### text cleaning

In [5]:
import nltk
import re

# make all elements string
df_texts['report_text'] = df_texts['report_text'].astype(str)
# Remove null fields
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# Make all text lowercase
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: x.lower())
# Delete stop-words => to be tesred later
#stopwords = nltk.corpus.stopwords.words('french')


In [6]:
all_texts = df_texts['report_text'].to_list()[500:]
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2297


### Création de LMDataBunch

In [7]:
all_texts[:10]

["oïdium-les preüiers symptômes naladie viennent d'apparaître ceps atteints 1966. pour prévenir attaques, premier traitenent devrait être appliqué, semaine prochaine, dans foyers l'an dernier cépages sensibles. bordeaux, avril 1967 contrôleur chargé avertissements roussel l'inspecteur protection végétaux bruneteau",
 'bulletin mars 1979 tous departements ..........................',
 'arbres fruitiers',
 "chancre commun pommier -dans vergers situés dans expositions très favorables cette maladie, variétés sensibles, conseillé d'exécuter pulvérisation cuprique lorsque arbres auront atteint stade fleckinger (premier gonflement bourgeon, écailles s'écartant légèrement).",
 "anthonome pommier -très localement, constate recrudescence l'activité insecte occasionné,en 1978, dégâts, parfois importants, dans quelques rares vergers. dans vergers pommiers l'anthonome s'est manifesté printemps dernier, conseillé d'effectuer traitement insecticide lcrsque arbres auront atteint stade (premier gonflem

In [8]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=4, #was 16, even 8 won't do
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [ ]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cpu,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=False) #was true with gpu

In [11]:
lm_learner.fit(epochs=10, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(710, 1.9302346756760502)

In [12]:
lm_learner.validate()

{'loss': 0.10752222873270512, 'perplexity': 1.1135156154632568}

In [13]:
lm_learner.save_model()

In [14]:
del lm_learner

### Création de databunch pour la classification

In [4]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [5]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [6]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cpu, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=False,#True when is cuda
                        logging_steps=50)

Some weights of the model checkpoint at model/model_out were not used when initializing CamembertForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForMultiLabelSequenceClassification were not initialized from the model checkpoint at model/model_out and are newly initialized: ['classifier.dense.w

In [19]:
cl_learner.fit(epochs=10,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

(4310, 0.18442438038751016)

In [20]:
cl_learner.validate()

{'loss': 0.5417979303057547,
 'fbeta': 0.38074934482574463,
 'roc_auc': 0.9135069592292953}

In [21]:
cl_learner.save_model()

In [22]:
del cl_learner

In [8]:
cl_learner.model.eval()

CamembertForMultiLabelSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

### Prédictions

In [23]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [24]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.9932581186294556), ('disease', 0.008345227688550949)]

In [25]:
predictor.predict("election américane")

[('bioagressor', 0.005320430733263493), ('disease', 0.0025872974656522274)]

In [134]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

[('disease', 0.9940599203109741), ('bioagressor', 0.01853816956281662)]

In [27]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('bioagressor', 0.005786733236163855), ('disease', 0.002268698997795582)]

In [133]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique => good
predictor.predict("mouche de la carotte :ajouter trichloronate")

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.0050725447945296764), ('disease', 0.0025651054456830025)]

In [29]:
#cas disease: 0, bioagressor: 0 - oidium : pas d'intervention => pas bon
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.9907628893852234), ('bioagressor', 0.010729578323662281)]

In [31]:
#cas disease: 0, bioagressor: 0 - mildiou : pas d'intervention :=> pas bon
predictor.predict("mildiou : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.9871827363967896), ('bioagressor', 0.008436040952801704)]

In [30]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.9874699711799622), ('bioagressor', 0.007616541814059019)]

In [32]:
#cas disease: 0, bioagressor: 0 - pucerons- le seuil de nuisibilité  n’est pas atteint => raté
predictor.predict("pucerons les pucerons verts sont toujours présents dans certaines parcelles. on note également la présence de sitobion avenae, relativement moins fréquente, mais avec des colonies de plusieurs dizaines d'individus. le temps frais actuel n'est pas favorable aux prédateurs, mais on observe un nombre important de pucerons «momifiés» par la ponte de parasitoïdes. le seuil de nuisibilité n'est atteint dans aucune parcelle du réseau")

[('bioagressor', 0.993032693862915), ('disease', 0.007686650846153498)]

In [33]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('bioagressor', 0.005601678974926472), ('disease', 0.0022960256319493055)]

In [34]:
#cas disease: 0, bioagressor: 0 news + mildiou => raté
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.987871527671814), ('bioagressor', 0.008575997315347195)]

In [35]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

[('bioagressor', 0.008640721440315247), ('disease', 0.001558510004542768)]

In [36]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('bioagressor', 0.006738507188856602), ('disease', 0.001968122087419033)]

In [37]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('bioagressor', 0.003678131615743041), ('disease', 0.003635833039879799)]

In [38]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

[('disease', 0.9834801554679871), ('bioagressor', 0.006759047508239746)]

In [41]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse => raté, il n'a pas encore vu sécheresse, mais ok pour jaunisse ou pourriture
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('disease', 0.939736008644104), ('bioagressor', 0.0030058857519179583)]

In [42]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

[('bioagressor', 0.9914067983627319), ('disease', 0.006071482319384813)]

In [50]:
#cas disease: 1, bioagressor: 1 - Carpocapse, tavelure
predictor.predict("Favorisée par les conditions climatiques,l'activité du Carpocapse est toujours intense et il est conseillé de renouveler la protection des fruits en effectuant un nouveau traitement dès réception de ce bulletin. Tavelure du pommier: la maturité des périthèces de cette tavelure est très avancés et les prochaines pluies risquent d'être contaminatrices")

[('bioagressor', 0.9982835054397583), ('disease', 0.767039954662323)]

In [49]:
#cas disease: 1, bioagressor: 0 tavelure
predictor.predict("tavelure du pommier: la maturité des périthèces de cette tavelure est très avancés et les prochaines pluies risquent d'être contaminatrices")

[('disease', 0.9612712860107422), ('bioagressor', 0.0036171094980090857)]

### Analyse the fine-tuned model

In [137]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [138]:
#evaluation on validation set
df_val = pd.read_csv('./data/val_set.csv')

#predictor.get_learner()

In [139]:
df_val.Disease = df_val.Disease.astype(int)
df_val.tail()

,Unnamed: 0,report_text,source_name,Bioagressor,Disease
75,247,doryphore -traiter les larves encore jeunes av...,AA_TC-PACA_1980_014,1,0
76,228,"~ doryphore, :au lieu de carbaryl 100 gr. de m...",AA_TC_Champagne-Ardenne_1965_003,0,0
77,371,tavelures des poiriers et des pommiers -corne ...,AA_TC_Aquitaine_1967_013,0,1
78,176,desherbage du tournesolcette culture est très ...,AA_GC_Aquitaine_1992_005,0,0
79,272,mais doux pyrale :\r\nl'activité de nymphose e...,AA_CL_Centre_1994_018,1,0


In [140]:
batch_predictions = predictor.predict_batch(df_val.report_text.to_list())

/home/shufan/tools/anaconda3/envs/fastCmb/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [141]:
batch_predictions[:10]

[[('disease', 0.997979462146759), ('bioagressor', 0.1519436240196228)],
 [('bioagressor', 0.004679505713284016), ('disease', 0.002755467314273119)],
 [('bioagressor', 0.9981618523597717), ('disease', 0.8399409055709839)],
 [('bioagressor', 0.03359901160001755), ('disease', 0.0007029788685031235)],
 [('bioagressor', 0.005871049594134092), ('disease', 0.002367417560890317)],
 [('disease', 0.0378008596599102), ('bioagressor', 0.0014056613435968757)],
 [('bioagressor', 0.005548827815800905), ('disease', 0.0023640908766537905)],
 [('bioagressor', 0.8302687406539917), ('disease', 0.0007327529601752758)],
 [('bioagressor', 0.007630123291164637), ('disease', 0.0017086350126191974)],
 [('bioagressor', 0.00813988596200943), ('disease', 0.0016845848876982927)]]

In [142]:
#dict(batch_predictions[1]).values()
list_y_pred = [ dict(pred) for pred in batch_predictions]
#list_y_pred[-5:]

In [143]:
df_y_pred = pd.DataFrame(list_y_pred, columns =['bioagressor', 'disease']) 
df_y_pred = df_y_pred.rename(columns={"bioagressor": "Bioagressor", "disease": "Disease"})
df_y_pred.tail()

,Bioagressor,Disease
75,0.993308,0.008235
76,0.987378,0.004707
77,0.001178,0.061532
78,0.005776,0.002200
79,0.994049,0.009508


In [144]:
df_y_pred.describe()

,Bioagressor,Disease
count,80.000000,80.000000
mean,0.339233,0.213201
std,0.461563,0.393560
min,0.001132,0.000506
25%,0.005121,0.002367
50%,0.008038,0.004257
75%,0.989209,0.034606
max,0.998390,0.997979


In [145]:
df_y_real = pd.DataFrame(df_val, columns=['Bioagressor', 'Disease'])
df_y_real.tail()

,Bioagressor,Disease
75,1,0
76,0,0
77,0,1
78,0,0
79,1,0


In [146]:
df_y_real.describe()

,Bioagressor,Disease
count,80.000000,80.000000
mean,0.237500,0.275000
std,0.428236,0.449331
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [147]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [148]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None) #If None, the scores for each class are returned

array([0.62962963, 0.72222222])

In [149]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5, average='weighted')

0.6793134598012648

In [150]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.89473684, 0.59090909])

In [151]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.7317073170731707

In [152]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.73913043, 0.65      ])

In [153]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.691304347826087

In [154]:
accuracy_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5)

0.6875

In [155]:
accuracy_score(y_true=df_y_real.Bioagressor.values, y_pred=df_y_pred.Bioagressor.values > 0.5)


0.85

In [156]:
accuracy_score(y_true=df_y_real.Disease.values, y_pred=df_y_pred.Disease.values > 0.5 )


0.825

In [157]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average=None)

array([0.60563045, 0.48902959])

In [158]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average='weighted')

0.5430641352435323

In [165]:
df_results = pd.concat([df_val, df_y_pred > 0.5], axis=1, ignore_index=True)

In [162]:
df_results.to_csv('predictions.csv')